# Full Scale Sentiment Analysis (Stateless & Robust)
## **Course:** Advanced NLP (Fall 2025)

## This notebook implements a production-grade pipeline designed for large datasets (950k+ posts).
## Key Features:
### 1. **Stateless Inference:** Each post is processed independently. The LLM retains zero memory of previous posts.
### 2. **Robust Checkpointing:** Results are saved incrementally. If the system crashes, execution resumes automatically from the last saved row.
### 3. **Memory Efficient:** Processes data in batches to respect RAM limits.

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ollama
from tqdm import tqdm

## Add scripts path for modular imports

In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from scripts.preprocessor import TextPreprocessor

## Visualization Style

In [ ]:
sns.set_theme(style="whitegrid")

## 1. Configuration

### Paths

In [ ]:
DATA_DIR = "../data"
PROCESSED_DIR = "../data/processed"
RESULTS_DIR = "../results"

### Create directories if they don't exist

In [ ]:
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

### Files

In [ ]:
MASTER_DATA_FILE = os.path.join(PROCESSED_DIR, "master_cleaned_dataset.csv")
FINAL_RESULTS_FILE = os.path.join(PROCESSED_DIR, "final_sentiment_results.csv")

In [ ]:
MODEL_NAME = "llama3"
BATCH_SIZE = 100  # Save to disk every 100 posts
SOURCES = ["kafiha", "TweetyChannel", "radiofarda", "iranintlTV", "bbcpersian"]

## 2. Data Preparation (Load, Clean, Filter)
### To ensure indices align perfectly for resuming, we first create a **Master Cleaned Dataset**.
### If this file already exists, we skip this step to save time.

In [ ]:
if os.path.exists(MASTER_DATA_FILE):
    print(f"✅ Master dataset found at {MASTER_DATA_FILE}. Loading...")
    full_df = pd.read_csv(MASTER_DATA_FILE)
else:
    print("⚠️ Master dataset not found. Building from scratch...")
    
    # 1. Load Raw Data
    all_dfs = []
    for source in SOURCES:
        path = os.path.join(DATA_DIR, f"{source}_messages.csv")
        if os.path.exists(path):
            print(f"   - Loading {source}...")
            temp_df = pd.read_csv(path)
            temp_df['source'] = source
            all_dfs.append(temp_df)
    
    if not all_dfs:
        raise FileNotFoundError("No source CSV files found in data/ directory!")
        
    full_df = pd.concat(all_dfs, ignore_index=True)
    
    # 2. Preprocessing
    print("   - Initializing Preprocessor...")
    preprocessor = TextPreprocessor()
    
    # Fill NaNs
    full_df['text'] = full_df['text'].fillna('')
    full_df['emoji_reactions'] = full_df['emoji_reactions'].fillna('')
    
    print("   - Cleaning text (This might take a few minutes for 900k+ rows)...")
    tqdm.pandas(desc="Cleaning")
    full_df['clean_post_text'] = full_df['text'].progress_apply(preprocessor.clean_text)
    full_df['clean_reactions'] = full_df['emoji_reactions'].apply(preprocessor.clean_reactions)
    
    # 3. Filtering (Remove short/empty texts)
    initial_len = len(full_df)
    full_df = full_df[full_df['clean_post_text'].str.len() > 10].reset_index(drop=True)
    print(f"   - Filtered {initial_len - len(full_df)} short/empty posts.")
    
    # 4. Save Master File
    print(f"   - Saving Master Dataset ({len(full_df)} rows)...")
    full_df.to_csv(MASTER_DATA_FILE, index=False, encoding='utf-8-sig')

print(f"🚀 Ready to process {len(full_df)} posts.")

## 3. Stateless Inference Engine
### We define the logic to query Llama 3.
### **Important:** `messages` list is recreated inside the function every time to ensure **Zero Memory** of previous posts.

In [ ]:
def get_sentiment_label(text, reactions):
    """
    Stateless Sentiment Analysis using Few-Shot Prompting.
    Each call is independent. No context is shared between posts.
    """
    prompt_text = f"متن پست: {text}\nواکنش‌های کاربران: {reactions}"
    
    # Optimized Few-Shot Prompt
    system_prompt = (
        "تو یک تحلیلگر دقیق احساسات فارسی هستی. بر اساس مثال‌های زیر، پست جدید را طبقه‌بندی کن.\n"
        "دسته‌بندی‌ها: ['خوشحال', 'ناراحت', 'عصبانی', 'مضطرب', 'خنثی', 'نگران']\n\n"
        "--- مثال‌های آموزشی ---\n"
        "مثال ۱ (تبلیغات/خبر): تور استانبول ویژه بلک فرایدی. -> خنثی\n"
        "مثال ۲ (غم/دلتنگی): فراموش کردنت سخته عزیزم. -> ناراحت\n"
        "مثال ۳ (طنز/موفقیت): وای چقدر خوشحالم تموم شد. -> خوشحال\n"
        "مثال ۴ (خشم/اعتراض): چرا وضعیت اینترنت اینطوریه؟ -> عصبانی\n"
        "مثال ۵ (نگرانی/پول): آخر ماه شد و حقوق نریختن. -> نگران\n"
        "--- پایان مثال‌ها ---\n"
        "فقط برچسب را بنویس."
    )

    try:
        # Generate a fresh request (Stateless)
        response = ollama.chat(model=MODEL_NAME, messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': prompt_text},
        ])
        
        label = response['message']['content'].strip()
        
        # Basic Validation
        valid_labels = ['خوشحال', 'ناراحت', 'عصبانی', 'مضطرب', 'خنثی', 'نگران']
        for v in valid_labels:
            if v in label: return v
        return "خنثی" # Fallback
    except Exception as e:
        return "Error"

## 4. Main Execution Loop (With Resume Capability)
### We check `final_sentiment_results.csv`. If it exists, we count the rows and skip that many in `full_df`.

#### 1. Check Progress

In [ ]:
if os.path.exists(FINAL_RESULTS_FILE):
    # Count lines in existing file to determine start index
    # We substract 1 for the header
    with open(FINAL_RESULTS_FILE, 'r', encoding='utf-8-sig') as f:
        processed_count = sum(1 for _ in f) - 1
    
    # Determine mode
    write_mode = 'a' # Append
    header = False   # Don't write header again
    start_index = processed_count
    print(f"🔄 Resuming from index {start_index}...")
else:
    write_mode = 'w' # Write new
    header = True    # Write header
    start_index = 0
    print("🟢 Starting new analysis...")

#### 2. Inference Loop

In [ ]:
if start_index < len(full_df):
    batch_buffer = []
    
    # Iterate from the last checkpoint
    for i in tqdm(range(start_index, len(full_df)), desc="Processing", initial=start_index, total=len(full_df)):
        
        row = full_df.iloc[i]
        
        # Call LLM (Stateless)
        sentiment = get_sentiment_label(row['clean_post_text'], row['clean_reactions'])
        
        # Prepare Result Row
        result_dict = {
            'msg_id': row.get('msg_id'),
            'date': row.get('date'),
            'source': row.get('source'),
            'clean_text': row['clean_post_text'], # Saving cleaned text to save space
            'sentiment': sentiment
        }
        batch_buffer.append(result_dict)
        
        # 3. Save to Disk (Batch)
        if len(batch_buffer) >= BATCH_SIZE:
            df_batch = pd.DataFrame(batch_buffer)
            df_batch.to_csv(FINAL_RESULTS_FILE, mode=write_mode, header=header, index=False, encoding='utf-8-sig')
            
            # Reset buffer and flags
            batch_buffer = []
            header = False # Never write header after first batch
            write_mode = 'a' # Always append hereafter
            
    # 4. Save remaining items
    if batch_buffer:
        df_batch = pd.DataFrame(batch_buffer)
        df_batch.to_csv(FINAL_RESULTS_FILE, mode=write_mode, header=header, index=False, encoding='utf-8-sig')

    print("\n✅ Analysis Complete! All rows processed.")
else:
    print("\n✅ All rows were already processed!")

### 5. Visualization & Statistics
#### Now we load the FINAL processed file and generate the report.

In [ ]:
print("Loading final results for visualization...")
df_results = pd.read_csv(FINAL_RESULTS_FILE)
df_results['date'] = pd.to_datetime(df_results['date'])
df_results['month_year'] = df_results['date'].dt.to_period('M')

## --- A. Trend Analysis (Time Series) ---
### Group by Month and Sentiment

In [ ]:
trend_data = df_results.groupby(['month_year', 'sentiment']).size().unstack(fill_value=0)

#### Filter for top 4 active sentiments

In [ ]:
target_cols = ['خوشحال', 'ناراحت', 'عصبانی', 'نگران']
existing_cols = [c for c in target_cols if c in trend_data.columns]

plt.figure(figsize=(14, 7))
trend_data[existing_cols].plot(kind='line', marker='o', figsize=(14, 7))
plt.title("Sentiment Trends Over 5 Years (Monthly)")
plt.xlabel("Date")
plt.ylabel("Post Count")
plt.legend(title="Sentiment")
plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(RESULTS_DIR, "final_trend_analysis.png"))
plt.show()

## --- B. Hope vs. Despair (Ratio) ---

In [ ]:
def categorize_mood(sent):
    if sent == 'خوشحال': return 'Hope/Positive'
    if sent in ['ناراحت', 'عصبانی', 'نگران', 'مضطرب']: return 'Despair/Negative'
    return 'Neutral'

df_results['mood_category'] = df_results['sentiment'].apply(categorize_mood)

### Calculate percentages per source

In [ ]:
mood_stats = df_results.groupby(['source', 'mood_category']).size().unstack(fill_value=0)
mood_stats['Total'] = mood_stats.sum(axis=1)
mood_stats['Despair_Rate'] = (mood_stats.get('Despair/Negative', 0) / mood_stats['Total'] * 100).round(1)

print("\n--- Mood Statistics per Channel ---")
display(mood_stats)
mood_stats.to_csv(os.path.join(RESULTS_DIR, "final_mood_statistics.csv"))

### Bar Chart

In [ ]:
mood_stats[['Hope/Positive', 'Despair/Negative', 'Neutral']].plot(kind='bar', stacked=True, color=['green', 'red', 'gray'], figsize=(10, 6))
plt.title("Sentiment Distribution by Channel")
plt.ylabel("Number of Posts")
plt.xticks(rotation=0)
plt.savefig(os.path.join(RESULTS_DIR, "final_mood_barchart.png"))
plt.show()